# Double Pendulum Chaos

## Introduction

The double pendulum is a classic example of a **chaotic dynamical system** in classical mechanics. Despite being governed by deterministic equations, the system exhibits extreme sensitivity to initial conditions—a hallmark of chaos. This notebook derives the equations of motion using the Lagrangian formalism and numerically integrates them to visualize the chaotic dynamics.

## Physical Setup

Consider two point masses $m_1$ and $m_2$ connected by rigid, massless rods of lengths $L_1$ and $L_2$. The first pendulum is attached to a fixed pivot, and the second pendulum hangs from the first mass. We denote the angles from the vertical as $\theta_1$ and $\theta_2$.

## Mathematical Derivation

### Cartesian Coordinates

The positions of the two masses in Cartesian coordinates are:

$$x_1 = L_1 \sin\theta_1, \quad y_1 = -L_1 \cos\theta_1$$

$$x_2 = L_1 \sin\theta_1 + L_2 \sin\theta_2, \quad y_2 = -L_1 \cos\theta_1 - L_2 \cos\theta_2$$

### Kinetic and Potential Energy

The velocities are obtained by differentiation:

$$\dot{x}_1 = L_1 \dot{\theta}_1 \cos\theta_1, \quad \dot{y}_1 = L_1 \dot{\theta}_1 \sin\theta_1$$

$$\dot{x}_2 = L_1 \dot{\theta}_1 \cos\theta_1 + L_2 \dot{\theta}_2 \cos\theta_2$$
$$\dot{y}_2 = L_1 \dot{\theta}_1 \sin\theta_1 + L_2 \dot{\theta}_2 \sin\theta_2$$

The kinetic energy is:

$$T = \frac{1}{2}m_1(\dot{x}_1^2 + \dot{y}_1^2) + \frac{1}{2}m_2(\dot{x}_2^2 + \dot{y}_2^2)$$

Expanding:

$$T = \frac{1}{2}(m_1 + m_2)L_1^2\dot{\theta}_1^2 + \frac{1}{2}m_2 L_2^2\dot{\theta}_2^2 + m_2 L_1 L_2 \dot{\theta}_1 \dot{\theta}_2 \cos(\theta_1 - \theta_2)$$

The potential energy (taking the pivot as reference):

$$V = m_1 g y_1 + m_2 g y_2 = -(m_1 + m_2)g L_1 \cos\theta_1 - m_2 g L_2 \cos\theta_2$$

### Lagrangian

The Lagrangian is $\mathcal{L} = T - V$:

$$\mathcal{L} = \frac{1}{2}(m_1 + m_2)L_1^2\dot{\theta}_1^2 + \frac{1}{2}m_2 L_2^2\dot{\theta}_2^2 + m_2 L_1 L_2 \dot{\theta}_1 \dot{\theta}_2 \cos(\theta_1 - \theta_2)$$
$$+ (m_1 + m_2)g L_1 \cos\theta_1 + m_2 g L_2 \cos\theta_2$$

### Equations of Motion

Applying the Euler-Lagrange equations $\frac{d}{dt}\frac{\partial \mathcal{L}}{\partial \dot{\theta}_i} - \frac{\partial \mathcal{L}}{\partial \theta_i} = 0$, we obtain the coupled second-order ODEs:

$$\ddot{\theta}_1 = \frac{-g(2m_1 + m_2)\sin\theta_1 - m_2 g \sin(\theta_1 - 2\theta_2) - 2\sin(\theta_1 - \theta_2)m_2(\dot{\theta}_2^2 L_2 + \dot{\theta}_1^2 L_1 \cos(\theta_1 - \theta_2))}{L_1(2m_1 + m_2 - m_2\cos(2\theta_1 - 2\theta_2))}$$

$$\ddot{\theta}_2 = \frac{2\sin(\theta_1 - \theta_2)(\dot{\theta}_1^2 L_1(m_1 + m_2) + g(m_1 + m_2)\cos\theta_1 + \dot{\theta}_2^2 L_2 m_2 \cos(\theta_1 - \theta_2))}{L_2(2m_1 + m_2 - m_2\cos(2\theta_1 - 2\theta_2))}$$

These equations are **nonlinear** and **coupled**, making analytical solutions intractable. We must resort to numerical integration.

## Numerical Implementation

We convert the system to four first-order ODEs by introducing $\omega_1 = \dot{\theta}_1$ and $\omega_2 = \dot{\theta}_2$, then integrate using `scipy.integrate.solve_ivp`.

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib.animation as animation

# Physical parameters
g = 9.81      # gravitational acceleration (m/s^2)
L1 = 1.0      # length of pendulum 1 (m)
L2 = 1.0      # length of pendulum 2 (m)
m1 = 1.0      # mass of pendulum 1 (kg)
m2 = 1.0      # mass of pendulum 2 (kg)

def double_pendulum_derivatives(t, state):
    """
    Compute the derivatives for the double pendulum system.
    
    Parameters:
    -----------
    t : float
        Time (not used explicitly as system is autonomous)
    state : array-like
        State vector [theta1, omega1, theta2, omega2]
    
    Returns:
    --------
    derivatives : list
        [dtheta1/dt, domega1/dt, dtheta2/dt, domega2/dt]
    """
    theta1, omega1, theta2, omega2 = state
    
    delta = theta1 - theta2
    
    # Denominator (common to both equations)
    den1 = L1 * (2*m1 + m2 - m2*np.cos(2*delta))
    den2 = L2 * (2*m1 + m2 - m2*np.cos(2*delta))
    
    # Angular acceleration of first pendulum
    domega1 = (-g*(2*m1 + m2)*np.sin(theta1) 
               - m2*g*np.sin(theta1 - 2*theta2) 
               - 2*np.sin(delta)*m2*(omega2**2*L2 + omega1**2*L1*np.cos(delta))) / den1
    
    # Angular acceleration of second pendulum
    domega2 = (2*np.sin(delta)*(omega1**2*L1*(m1 + m2) 
               + g*(m1 + m2)*np.cos(theta1) 
               + omega2**2*L2*m2*np.cos(delta))) / den2
    
    return [omega1, domega1, omega2, domega2]

print("Double pendulum derivatives function defined.")

## Demonstrating Sensitivity to Initial Conditions

The essence of chaos is **sensitive dependence on initial conditions**. We simulate two nearly identical initial conditions and observe how their trajectories diverge exponentially over time.

In [ ]:
# Time span and evaluation points
t_span = (0, 30)  # 30 seconds
t_eval = np.linspace(0, 30, 3000)

# Initial conditions: [theta1, omega1, theta2, omega2]
# Pendulum 1: Starting at 120 degrees for both angles
theta1_0 = np.radians(120)
theta2_0 = np.radians(120)
omega1_0 = 0.0
omega2_0 = 0.0

initial_state_1 = [theta1_0, omega1_0, theta2_0, omega2_0]

# Pendulum 2: Perturbed by 0.1 degrees (~0.00175 radians)
perturbation = np.radians(0.1)
initial_state_2 = [theta1_0 + perturbation, omega1_0, theta2_0, omega2_0]

# Solve the ODEs
solution_1 = solve_ivp(double_pendulum_derivatives, t_span, initial_state_1, 
                       t_eval=t_eval, method='RK45', rtol=1e-10, atol=1e-12)
solution_2 = solve_ivp(double_pendulum_derivatives, t_span, initial_state_2, 
                       t_eval=t_eval, method='RK45', rtol=1e-10, atol=1e-12)

print(f"Integration complete. Solution shapes: {solution_1.y.shape}")
print(f"Initial perturbation: {perturbation:.6f} radians ({np.degrees(perturbation):.2f} degrees)")

## Computing Cartesian Trajectories

We convert from generalized coordinates $(\theta_1, \theta_2)$ to Cartesian coordinates $(x, y)$ for visualization.

In [ ]:
def compute_positions(theta1, theta2):
    """
    Compute Cartesian positions of both pendulum masses.
    
    Parameters:
    -----------
    theta1 : array-like
        Angle of first pendulum from vertical
    theta2 : array-like
        Angle of second pendulum from vertical
    
    Returns:
    --------
    x1, y1, x2, y2 : arrays
        Cartesian coordinates of masses
    """
    x1 = L1 * np.sin(theta1)
    y1 = -L1 * np.cos(theta1)
    x2 = x1 + L2 * np.sin(theta2)
    y2 = y1 - L2 * np.cos(theta2)
    return x1, y1, x2, y2

# Compute positions for both pendulums
x1_1, y1_1, x2_1, y2_1 = compute_positions(solution_1.y[0], solution_1.y[2])
x1_2, y1_2, x2_2, y2_2 = compute_positions(solution_2.y[0], solution_2.y[2])

print("Cartesian coordinates computed for both trajectories.")

## Visualization of Chaotic Behavior

We create a comprehensive visualization showing:
1. The trajectories of the second mass for both initial conditions
2. The divergence of angle $\theta_1$ over time
3. Phase space portrait
4. The growth of separation between trajectories

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(14, 10))

# Plot 1: Trajectory of second mass (tip of double pendulum)
ax1 = fig.add_subplot(2, 2, 1)
ax1.plot(x2_1, y2_1, 'b-', alpha=0.6, linewidth=0.5, label='Original IC')
ax1.plot(x2_2, y2_2, 'r-', alpha=0.6, linewidth=0.5, label='Perturbed IC')
ax1.set_xlabel('x (m)', fontsize=11)
ax1.set_ylabel('y (m)', fontsize=11)
ax1.set_title('Trajectory of Second Mass', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right', fontsize=9)
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)

# Plot 2: Angular displacement over time
ax2 = fig.add_subplot(2, 2, 2)
ax2.plot(solution_1.t, np.degrees(solution_1.y[0]), 'b-', linewidth=0.8, label='Original IC')
ax2.plot(solution_2.t, np.degrees(solution_2.y[0]), 'r-', linewidth=0.8, label='Perturbed IC')
ax2.set_xlabel('Time (s)', fontsize=11)
ax2.set_ylabel(r'$\theta_1$ (degrees)', fontsize=11)
ax2.set_title('Angular Displacement of First Pendulum', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right', fontsize=9)
ax2.grid(True, alpha=0.3)

# Plot 3: Phase space portrait (theta1 vs omega1)
ax3 = fig.add_subplot(2, 2, 3)
ax3.plot(np.degrees(solution_1.y[0]), solution_1.y[1], 'b-', alpha=0.6, linewidth=0.5, label='Original IC')
ax3.plot(np.degrees(solution_2.y[0]), solution_2.y[1], 'r-', alpha=0.6, linewidth=0.5, label='Perturbed IC')
ax3.set_xlabel(r'$\theta_1$ (degrees)', fontsize=11)
ax3.set_ylabel(r'$\omega_1$ (rad/s)', fontsize=11)
ax3.set_title('Phase Space Portrait', fontsize=12, fontweight='bold')
ax3.legend(loc='upper right', fontsize=9)
ax3.grid(True, alpha=0.3)

# Plot 4: Separation growth (indicator of chaos)
ax4 = fig.add_subplot(2, 2, 4)

# Compute angular separation
delta_theta1 = np.abs(solution_1.y[0] - solution_2.y[0])
delta_theta2 = np.abs(solution_1.y[2] - solution_2.y[2])

# Euclidean distance between second masses
separation = np.sqrt((x2_1 - x2_2)**2 + (y2_1 - y2_2)**2)

ax4.semilogy(solution_1.t, separation, 'g-', linewidth=1.0)
ax4.axhline(y=perturbation, color='k', linestyle='--', alpha=0.5, label='Initial perturbation')
ax4.set_xlabel('Time (s)', fontsize=11)
ax4.set_ylabel('Separation (m)', fontsize=11)
ax4.set_title('Divergence of Trajectories (Log Scale)', fontsize=12, fontweight='bold')
ax4.legend(loc='lower right', fontsize=9)
ax4.grid(True, alpha=0.3, which='both')
ax4.set_ylim([1e-6, 10])

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved as 'plot.png'")

## Energy Conservation Check

In the absence of friction, the total mechanical energy $E = T + V$ should be conserved. We verify this as a check on our numerical integration.

In [ ]:
def compute_energy(theta1, omega1, theta2, omega2):
    """
    Compute total mechanical energy of the double pendulum.
    
    Returns:
    --------
    T : array
        Kinetic energy
    V : array
        Potential energy
    E : array
        Total energy
    """
    # Kinetic energy
    T = (0.5*(m1 + m2)*L1**2*omega1**2 
         + 0.5*m2*L2**2*omega2**2 
         + m2*L1*L2*omega1*omega2*np.cos(theta1 - theta2))
    
    # Potential energy (reference at pivot)
    V = (-(m1 + m2)*g*L1*np.cos(theta1) 
         - m2*g*L2*np.cos(theta2))
    
    return T, V, T + V

# Compute energy for first trajectory
T, V, E = compute_energy(solution_1.y[0], solution_1.y[1], 
                         solution_1.y[2], solution_1.y[3])

# Check energy conservation
E_initial = E[0]
E_deviation = np.abs(E - E_initial) / np.abs(E_initial) * 100

print(f"Initial total energy: {E_initial:.6f} J")
print(f"Maximum energy deviation: {np.max(E_deviation):.6e} %")
print(f"Mean energy deviation: {np.mean(E_deviation):.6e} %")

# Plot energy components
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(solution_1.t, T, 'r-', label='Kinetic Energy', alpha=0.7)
ax.plot(solution_1.t, V, 'b-', label='Potential Energy', alpha=0.7)
ax.plot(solution_1.t, E, 'k-', label='Total Energy', linewidth=2)
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel('Energy (J)', fontsize=11)
ax.set_title('Energy Conservation in Double Pendulum', fontsize=12, fontweight='bold')
ax.legend(loc='right', fontsize=9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Lyapunov Exponent Estimation

The **Lyapunov exponent** $\lambda$ quantifies the rate of separation of infinitesimally close trajectories. For a chaotic system, $\lambda > 0$. We estimate it from our simulation:

$$\lambda \approx \frac{1}{t} \ln\left(\frac{\delta(t)}{\delta_0}\right)$$

where $\delta(t)$ is the separation at time $t$ and $\delta_0$ is the initial separation.

In [ ]:
# Estimate Lyapunov exponent from trajectory separation
# Use early-time behavior before saturation

# Find the time window where separation grows exponentially
# (before it saturates at system scale)
mask = (separation > 1e-5) & (separation < 1.0) & (solution_1.t > 0.1)

if np.sum(mask) > 10:
    t_fit = solution_1.t[mask]
    sep_fit = separation[mask]
    
    # Linear fit to log(separation) vs time
    log_sep = np.log(sep_fit)
    coeffs = np.polyfit(t_fit, log_sep, 1)
    lambda_est = coeffs[0]  # Slope is the Lyapunov exponent
    
    print(f"Estimated Lyapunov exponent: λ ≈ {lambda_est:.3f} s⁻¹")
    print(f"Positive Lyapunov exponent confirms chaotic behavior!")
    print(f"\nInterpretation: Nearby trajectories diverge by a factor of e every {1/lambda_est:.2f} seconds")
else:
    print("Insufficient data for Lyapunov exponent estimation.")

## Poincaré Section

A **Poincaré section** reduces the continuous phase space to a discrete map, revealing the underlying structure of chaotic dynamics. We sample the state whenever $\theta_2$ passes through zero with positive velocity.

In [ ]:
# Generate longer trajectory for Poincaré section
t_span_long = (0, 100)
t_eval_long = np.linspace(0, 100, 10000)

solution_long = solve_ivp(double_pendulum_derivatives, t_span_long, initial_state_1, 
                          t_eval=t_eval_long, method='RK45', rtol=1e-10, atol=1e-12)

# Find Poincaré section: theta2 crosses zero with positive velocity
theta2 = solution_long.y[2]
omega2 = solution_long.y[3]

# Detect zero crossings
poincare_indices = []
for i in range(1, len(theta2)):
    # Wrap theta2 to [-pi, pi]
    t2_prev = ((theta2[i-1] + np.pi) % (2*np.pi)) - np.pi
    t2_curr = ((theta2[i] + np.pi) % (2*np.pi)) - np.pi
    
    # Check for upward crossing through 0
    if t2_prev < 0 and t2_curr >= 0 and omega2[i] > 0:
        poincare_indices.append(i)

poincare_theta1 = solution_long.y[0][poincare_indices]
poincare_omega1 = solution_long.y[1][poincare_indices]

# Plot Poincaré section
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(np.degrees(poincare_theta1) % 360 - 180, poincare_omega1, 
           s=1, c='blue', alpha=0.6)
ax.set_xlabel(r'$\theta_1$ (degrees)', fontsize=11)
ax.set_ylabel(r'$\omega_1$ (rad/s)', fontsize=11)
ax.set_title('Poincaré Section (θ₂ = 0, ω₂ > 0)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Number of Poincaré points collected: {len(poincare_indices)}")

## Conclusion

This notebook demonstrates the chaotic behavior of the double pendulum system:

1. **Nonlinear coupling**: The equations of motion are highly nonlinear due to the trigonometric coupling between the two pendulum angles.

2. **Sensitivity to initial conditions**: A perturbation of just 0.1° leads to completely different trajectories after a few seconds, as shown by the exponential divergence plot.

3. **Positive Lyapunov exponent**: The estimated $\lambda > 0$ confirms chaotic dynamics.

4. **Energy conservation**: Despite chaotic behavior, total mechanical energy is conserved (within numerical precision), demonstrating that chaos ≠ randomness.

5. **Strange attractor structure**: The Poincaré section reveals the fractal-like structure underlying the chaotic motion.

The double pendulum serves as a powerful pedagogical example that deterministic systems can exhibit unpredictable behavior, with profound implications for weather forecasting, celestial mechanics, and other complex systems.